In [6]:
import pandas as pd

df = pd.read_csv("../../Data/m4_imputed.csv")


In [7]:
median_wl_time = df['wl_time'].median()
print(f"Median waiting list time: {median_wl_time}")

df['wl_time'] = df['wl_time'].apply(lambda x: 0 if x < median_wl_time else 1)
print(df['wl_time'].value_counts(normalize=True))

Median waiting list time: 78.0
1    0.501886
0    0.498114
Name: wl_time, dtype: float64


In [8]:
import keras.backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataClassifier
from sklearn.metrics import f1_score

X = df.drop(columns=['wl_time'])
y = df['wl_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = StructuredDataClassifier(max_trials=10,
                               project_name="m4_pre_feat",
                               metrics=["accuracy"],
                               overwrite=False)

clf.fit(X_train, y_train, verbose=1)
print(clf.evaluate(X_test, y_test))
f1_s = f1_score(y_test, clf.predict(X_test))
print(f"F1 score: {f1_s}")
print("F1 Macro:", f1_score(y_test, clf.predict(X_test), average='macro'))


Trial 10 Complete [00h 02m 23s]
val_accuracy: 0.6995954513549805

Best val_accuracy So Far: 0.7035598754882812
Total elapsed time: 00h 15m 09s
INFO:tensorflow:Oracle triggered exit
Epoch 1/28
1936/1936 [==============================] - 5s 2ms/step - loss: 0.6289 - accuracy: 0.6529
Epoch 2/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.6072 - accuracy: 0.6751
Epoch 3/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.6026 - accuracy: 0.6784
Epoch 4/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.5988 - accuracy: 0.6813
Epoch 5/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.5956 - accuracy: 0.6849
Epoch 6/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.5934 - accuracy: 0.6865
Epoch 7/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.5916 - accuracy: 0.6879
Epoch 8/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.5903 - accuracy: 0.6896
Epo

INFO:tensorflow:Assets written to: ./m4_pre_feat/best_model/assets


INFO:tensorflow:Assets written to: ./m4_pre_feat/best_model/assets


484/484 [==============================] - 1s 2ms/step - loss: 0.5978 - accuracy: 0.6865
[0.5977836847305298, 0.6865392327308655]
484/484 [==============================] - 1s 1ms/step
F1 score: 0.6575400465739891
484/484 [==============================] - 1s 1ms/step
F1 Macro: 0.6842752933399929
